In [ ]:
import os, sys
import pandas as pd
import matplotlib.pyplot as plt

from obspy import UTCDateTime

In [ ]:
if os.uname().nodename == "lighthouse":
    root_path = "/home/andbro/"
    data_path = "/home/andbro/kilauea-data/"
    archive_path = "/home/andbro/freenas/"
elif os.uname().nodename == "kilauea":
    root_path = "/home/brotzer/"
    data_path = "/import/kilauea-data/"
    archive_path = "/import/freenas-ffb-01-data/"

In [ ]:
config = {}

config['tbeg'] = "2023-07-28"
config['tend'] = "2023-08-14"

config['ring'] = "Z"

config['path_to_wromy_sds'] = archive_path+"romy_archive/"

config['path_to_data'] = archive_path+f"romy_autodata/2023/R{config['ring']}/"

In [ ]:
from functions.load_sagnac_beat_data import __load_sagnac_beat_data
from functions.smoothing import __smooth

from andbro__load_WROMY_stream import __load_wromy_stream
from andbro__load_FURT_stream import __load_furt_stream

In [ ]:
def __cumsum_rain(arr, xnum):

    from numpy import cumsum, zeros

    arr_out = zeros(len(arr))
    for i, dat in enumerate(arr):
        arr_out[i] = sum(arr[i:i+xnum])

    return arr_out

## Load Data

### Load Beat Data

In [ ]:
df = __load_sagnac_beat_data(config['tbeg'], config['tend'], config['ring'], config['path_to_data'])
df = df[df.times_mjd != 0]

In [ ]:
df.head()

### Load FURT Data

In [ ]:
furt = __load_furt_stream(config['tbeg'], config['tend'])

In [ ]:
config['seed_wromy'] = "BW.WROMY..WS8"

ws8 = __load_wromy_stream(config['path_to_wromy_sds'], config['seed_wromy'], config['tbeg'], config['tend'])

In [ ]:
def __makeplot_overview():

    from andbro__utc_to_mjd import __utc_to_mjd
    ref = __utc_to_mjd([config['tbeg']])

    N=5
    t_scale = 86400

    fig, ax = plt.subplots(N, 1, figsize=(15,10))

    ax[0].scatter(df['times_mjd']-ref, df['fz'], s=1, label=f"R{config['ring']}")
    # ax[0].plot(df['times_mjd'], df['fz'])

    ax[0].set_ylim(553.485, 553.5)

    ax[1].plot(furt.select(channel="*T")[0].times()/t_scale, furt.select(channel="*T")[0].data, color="tab:red", label="FURT-T")
    ax1_1 = ax[1].twinx()
    ax1_1.plot(ws8.select(channel="*T")[0].times()/t_scale, ws8.select(channel="*T")[0].data, label="WS8-T")

    ax[2].plot(furt.select(channel="*P")[0].times()/t_scale, furt.select(channel="*P")[0].data, label="Furt-P")

    ax[3].plot(furt.select(channel="*W")[0].times()/t_scale, __smooth(furt.select(channel="*W")[0].data, 300), label="FURT-wind speed (5min average)")
    ax[3].set_ylim(bottom=0)

    ax[4].plot(furt.select(channel="*D")[0].times()/t_scale, __smooth(furt.select(channel="*D")[0].data, 300), label="FURT-wind direction (5min average)")
    ax[4].set_ylim(0, 360)

    for i in range(N):
        ax[i].legend()

    ax[0].set_ylabel("Frequency (Hz)")
    ax[1].set_ylabel("Temperature (°C)")
    ax[2].set_ylabel("Pressure (hPa)")
    ax[3].set_ylabel("Wind Speed (m/s)")
    ax[4].set_ylabel("Wind Direction (°)")
    # ax[0].set_ylabel("")

    ax[N-1].set_xlabel(f"Time from {config['tbeg']} (days)")

    plt.show();

__makeplot_overview()

In [ ]:
path_to_data = archive_path+f"romy_autodata/2023/RV/"
df1 = pd.read_pickle(path_to_data+"FJV_20230801.pkl")
df1

In [ ]:
path_to_data = archive_path+f"romy_autodata/2023/RZ/"
df1 = pd.read_pickle(path_to_data+"FJZ_20230809.pkl")
df1